In [77]:
# Load libraries
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns
import optuna
from lifelines import KaplanMeierFitter, CoxPHFitter
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import Lasso, LassoCV
from sklearn.model_selection import KFold, train_test_split, GridSearchCV
from xgboost import XGBRegressor


In [36]:
# Load data
data = pd.read_csv("C:/Users/ashle/OneDrive/Documents/Courses/UNC Courses/BIOS 635 - ML/Midterm/train.csv")
data_test = pd.read_csv("C:/Users/ashle/OneDrive/Documents/Courses/UNC Courses/BIOS 635 - ML/Midterm/test.csv")

# Data Preprocessing 

In [21]:
# Examine data
print(data.shape)
print(data_test.shape)
data.head()

(28800, 60)
(3, 58)


,ID,dri_score,psych_disturb,cyto_score,diabetes,hla_match_c_high,hla_high_res_8,tbi_status,arrhythmia,hla_low_res_6,...,tce_div_match,donor_related,melphalan_dose,hla_low_res_8,cardiac,hla_match_drb1_high,pulm_moderate,hla_low_res_10,efs,efs_time
0,0,N/A - non-malignant indication,No,NaN,No,NaN,NaN,No TBI,No,6.0,...,NaN,Unrelated,"N/A, Mel not given",8.0,No,2.0,No,10.0,0.0,42.356
1,1,Intermediate,No,Intermediate,No,2.0,8.0,"TBI +- Other, >cGy",No,6.0,...,Permissive mismatched,Related,"N/A, Mel not given",8.0,No,2.0,Yes,10.0,1.0,4.672
2,2,N/A - non-malignant indication,No,NaN,No,2.0,8.0,No TBI,No,6.0,...,Permissive mismatched,Related,"N/A, Mel not given",8.0,No,2.0,No,10.0,0.0,19.793
3,3,High,No,Intermediate,No,2.0,8.0,No TBI,No,6.0,...,Permissive mismatched,Unrelated,"N/A, Mel not given",8.0,No,2.0,No,10.0,0.0,102.349
4,4,High,No,NaN,No,2.0,8.0,No TBI,No,6.0,...,Permissive mismatched,Related,MEL,8.0,No,2.0,No,10.0,0.0,16.223


Since there are so many NaNs, we will replace the missing data.

In [37]:
# Replace missing data

# Check missing values
print(f"Number of rows with NaN in Train: {data.isnull().any(axis=1).sum()}")

# Fill numerical variables with median
for col in data.select_dtypes(include=['number']).columns:  
    data[col].fillna(data[col].median(), inplace=True)

# Fill categorical variables with mode
for col in data.select_dtypes(include=['object', 'category']).columns:  
    data[col].fillna(data[col].mode()[0], inplace=True)
    
# Check NaNs are removed
print(f"Number of rows with NaN in Train now: {data.isnull().any(axis=1).sum()}")

# Do the same preprocessing for test set
print(f"Number of rows with NaN in Test: {data_test.isnull().any(axis=1).sum()}")

for col in data_test.select_dtypes(include=['number']).columns:  
    data_test[col].fillna(data_test[col].median(), inplace=True)

for col in data_test.select_dtypes(include=['object', 'category']).columns:  
    data_test[col].fillna(data_test[col].mode()[0], inplace=True)
    
print(f"Number of rows with NaN in Test now: {data_test.isnull().any(axis=1).sum()}")

Number of rows with NaN: 26826
Number of rows with NaN in Train now: 0
Number of rows with NaN in Test: 2
Number of rows with NaN in Test now: 0


In [24]:
# Define a function to calculate the survival rate
def transform_survival_rate(df, time_col='efs_time', event_col='efs'):
    """
    Transform the efs and efs_time data into survival rates using the Kaplan-Meier estimator.
    """
    kmf = KaplanMeierFitter()  # initalize estimator
    kmf.fit(df[time_col], df[event_col])  # fit the estimator
    survival_prob = kmf.survival_function_at_times(df[time_col]).to_numpy().flatten()  # calculate probabilites
    df["survival_rate"] = survival_prob  # add probabilities as new variable   
    return df

In [68]:
# Adding survival rate and risk score
data = transform_survival_rate(data, "efs_time", "efs")
data['risk_score'] = np.log(data['survival_rate'] / (1 - data['survival_rate']))

data.head()

,ID,dri_score,psych_disturb,cyto_score,diabetes,hla_match_c_high,hla_high_res_8,tbi_status,arrhythmia,hla_low_res_6,...,melphalan_dose,hla_low_res_8,cardiac,hla_match_drb1_high,pulm_moderate,hla_low_res_10,efs,efs_time,survival_rate,risk_score
0,0,N/A - non-malignant indication,No,Poor,No,2.0,8.0,No TBI,No,6.0,...,"N/A, Mel not given",8.0,No,2.0,No,10.0,0.0,42.356,0.458687,-0.165630
1,1,Intermediate,No,Intermediate,No,2.0,8.0,"TBI +- Other, >cGy",No,6.0,...,"N/A, Mel not given",8.0,No,2.0,Yes,10.0,1.0,4.672,0.847759,1.717134
2,2,N/A - non-malignant indication,No,Poor,No,2.0,8.0,No TBI,No,6.0,...,"N/A, Mel not given",8.0,No,2.0,No,10.0,0.0,19.793,0.462424,-0.150587
3,3,High,No,Intermediate,No,2.0,8.0,No TBI,No,6.0,...,"N/A, Mel not given",8.0,No,2.0,No,10.0,0.0,102.349,0.456661,-0.173792
4,4,High,No,Poor,No,2.0,8.0,No TBI,No,6.0,...,MEL,8.0,No,2.0,No,10.0,0.0,16.223,0.464674,-0.141542


In [41]:
features = data.drop(columns=['ID', 'efs', 'efs_time', 'survival_rate'])
features.head()

,dri_score,psych_disturb,cyto_score,diabetes,hla_match_c_high,hla_high_res_8,tbi_status,arrhythmia,hla_low_res_6,graft_type,...,karnofsky_score,hepatic_mild,tce_div_match,donor_related,melphalan_dose,hla_low_res_8,cardiac,hla_match_drb1_high,pulm_moderate,hla_low_res_10
0,N/A - non-malignant indication,No,Poor,No,2.0,8.0,No TBI,No,6.0,Bone marrow,...,90.0,No,Permissive mismatched,Unrelated,"N/A, Mel not given",8.0,No,2.0,No,10.0
1,Intermediate,No,Intermediate,No,2.0,8.0,"TBI +- Other, >cGy",No,6.0,Peripheral blood,...,90.0,No,Permissive mismatched,Related,"N/A, Mel not given",8.0,No,2.0,Yes,10.0
2,N/A - non-malignant indication,No,Poor,No,2.0,8.0,No TBI,No,6.0,Bone marrow,...,90.0,No,Permissive mismatched,Related,"N/A, Mel not given",8.0,No,2.0,No,10.0
3,High,No,Intermediate,No,2.0,8.0,No TBI,No,6.0,Bone marrow,...,90.0,Yes,Permissive mismatched,Unrelated,"N/A, Mel not given",8.0,No,2.0,No,10.0
4,High,No,Poor,No,2.0,8.0,No TBI,No,6.0,Peripheral blood,...,90.0,No,Permissive mismatched,Related,MEL,8.0,No,2.0,No,10.0


# Method 1: Lasso

In [60]:
# Check type of data structure
data.dtypes

ID                  int64
dri_score          object
psych_disturb      object
cyto_score         object
diabetes           object
                   ...   
pulm_moderate      object
hla_low_res_10    float64
efs               float64
efs_time          float64
survival_rate     float64
Length: 61, dtype: object

In [69]:
# Identify categorical variables
cat_cols = data.select_dtypes(include=['object']).columns.tolist()
print(len(cat_cols))

# One-hot encoding
fixed_data = pd.get_dummies(data, columns=cat_cols, drop_first=True)
fixed_data.head()

35


,ID,hla_match_c_high,hla_high_res_8,hla_low_res_6,hla_high_res_6,hla_high_res_10,hla_match_dqb1_high,hla_nmdp_6,hla_match_c_low,hla_match_drb1_low,...,tce_div_match_GvH non-permissive,tce_div_match_HvG non-permissive,tce_div_match_Permissive mismatched,donor_related_Related,donor_related_Unrelated,"melphalan_dose_N/A, Mel not given",cardiac_Not done,cardiac_Yes,pulm_moderate_Not done,pulm_moderate_Yes
0,0,2.0,8.0,6.0,6.0,10.0,2.0,6.0,2.0,2.0,...,0,0,1,0,1,1,0,0,0,0
1,1,2.0,8.0,6.0,6.0,10.0,2.0,6.0,2.0,2.0,...,0,0,1,1,0,1,0,0,0,1
2,2,2.0,8.0,6.0,6.0,10.0,2.0,6.0,2.0,2.0,...,0,0,1,1,0,1,0,0,0,0
3,3,2.0,8.0,6.0,6.0,10.0,2.0,6.0,2.0,2.0,...,0,0,1,0,1,1,0,0,0,0
4,4,2.0,8.0,6.0,6.0,10.0,2.0,5.0,2.0,2.0,...,0,0,1,1,0,0,0,0,0,0


In [79]:
# Split data into X and y
X = fixed_data.drop(columns=['ID','efs_time', 'efs', 'survival_rate', 'risk_score'])
y = fixed_data['risk_score']

# Split into 70-30 train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=7)

# Grid search tuning to find best lambda using cross validation
grid = 10 ** np.linspace(-4, 0, num=100)
lasso = LassoCV(alphas=grid, cv=5)
lasso.fit(X_train, y_train)

print(f"Best lambda: {lasso.alpha_}")

Best lambda: 0.0004037017258596554


In [ ]:
# Lasso Regression model
lasso = Lasso()

grid = 10 ** np.linspace(-4, 0, num=100)
grid_search = GridSearchCV(lasso, grid, cv=5)
grid_search.fit(X_train, y_train)

# Best Lasso model
best_lasso = grid_search.best_estimator_

# Predict risk scores
data['lasso_risk_score'] = best_lasso.predict(X)


# Method 2: XGBoost

# Method 3: Cox Regression